In [1]:
! pip install datasets transformers nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.7 MB/s eta 0:00:00


In [3]:
from transformers import (AutoModelForSeq2SeqLM, 
                            AutoTokenizer, 
                            T5Tokenizer)
import torch
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
from tqdm import tqdm

#import subprocess
import sys
import os
import argparse
from IPython.core import error
import random
import numpy as np
import nltk
import json
import csv

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
output_dir = '/content/drive/MyDrive/Colab Notebooks/Code/NLP_robotics/Shared_trained_models/t5-base-epoch20-infix-word-03-26-64220/' 
# here you need to link this path in your Google drive to the place preseving your model weights, e.g., checkpoint-72000

model_checkpoint = "t5-base"
prefix = "Transform the following sentence into Signal Temporal logic: "

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["logic_sentence"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    model_inputs["logic_sentence"] = examples["logic_sentence"]
    model_inputs["id"] = examples["id"]
    return model_inputs

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
tl_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir+"checkpoint-72000").to(device)

def NL2TL():
    output_list = []
    dataset = load_dataset('csv', data_files=output_dir + 'total_data.csv')
    test_dataset= dataset['train']
    tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)


    for i in range(len(tokenized_test_dataset)):
        inputs = [prefix + tokenized_test_dataset[i]['logic_sentence']]
        inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
        output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        output_list.append(decoded_output)
    return output_list


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [11]:
test_sentence = ['If (prop_1), then ensure that (prop_2) until (prop_3).',
                 'If (prop_1) afterward, (prop_2) and (prop_3) as well.',
                 'Start by (prop_1). Then, (prop_2). Lastly, (prop_3).',
                 'Do not (prop_1) at this point!', # Input the natural sentence
                 '(prop_1).',
                 'Sooner or later (prop_1)',
                 'Repeatedly (prop_1)',
                 'At some point, (prop_1).'] # Input the natural sentence


f = open(output_dir+'total_data.csv','w')
csv_writer = csv.writer(f)
csv_writer.writerow(['id', 'logic_ltl', 'logic_sentence'])

for i in range(len(test_sentence)):
  csv_writer.writerow([i, '', test_sentence[i]])
f.close()

In [12]:
translation = NL2TL()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a62c9a091f92e3f5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [13]:
for i in range(len(test_sentence)):
  print(test_sentence[i])
  print(translation[i] + '\n') # print the STL

If (prop_1), then ensure that (prop_2) until (prop_3).
( prop_1 imply ( prop_2 until prop_3 ) )

If (prop_1) afterward, (prop_2) and (prop_3) as well.
( prop_1 imply ( prop_2 and prop_3 ) )

Start by (prop_1). Then, (prop_2). Lastly, (prop_3).
( ( prop_1 until prop_2 ) until ( prop_3 ) )

Do not (prop_1) at this point!
negation ( prop_1 )

(prop_1).
prop_1

Sooner or later (prop_1)
finally ( prop_1 )

Repeatedly (prop_1)
globally ( finally prop_1 )

At some point, (prop_1).
finally ( prop_1 )

